# Bulk Alkanes

In this example, the MoSDeF toolkit is used to create a system of bulk alkanes, atom-typed according to the OPLS all-atom force field. The output files are then used to run an NPT molecular dynamics simulation using the GROMACS simulation engine.

Note: MDP files were obtained from https://github.com/mattwthompson/gromacs_signac_template/tree/master/src/util/mdp_files.

First, we import mBuild.

In [ ]:
import mbuild as mb

Here we define all of the necessary `Compound` classes for creating an alkane box. Our lowest level components here are a `CH2` moiety and a hydrogen atom. An `Alkane` class is defined that uses the `Polymer` recipe to stitch together a user-defined number of `CH2` moieties - which are then capped with hydrogen atoms to yield a linear alkane chain. The `AlkaneBox` class serves as a wrapper around mBuild's `fill_box` function and yields a cubic box of alkane chains.

In [ ]:
class CH2(mb.Compound):
    """A methylene bridge. """
    def __init__(self):
        super(CH2, self).__init__()
        
        mb.load('pdb-files/ch2.pdb', compound=self)
        self.add(mb.Port(anchor=self[0], orientation=[0, 1, 0], separation=0.075), label='up')
        self.add(mb.Port(anchor=self[0], orientation=[0, -1, 0], separation=0.075), label='down')
        
class H(mb.Compound):
    """A hydrogen atom. """
    def __init__(self):
        super(H, self).__init__()
        
        self.add(mb.Particle(name='H'))
        self.add(mb.Port(anchor=self[0], orientation=[0, 1, 0], separation=0.075), label='up')
        
class Alkane(mb.Compound):
    """A linear alkane chain. """
    def __init__(self, n):
        """Initialize an Alkane Compound.

        Parameters
        ----------
        n : int
            Number of carbon atoms in the chain
            
        """
        if n < 1:
            raise ValueError('n must be 1 or more')
        super(Alkane, self).__init__()

        chain = mb.Polymer(CH2(), n=n, port_labels=('up', 'down'))
        self.add(chain, 'chain')
        
        up_cap = H()
        down_cap = H()
        mb.force_overlap(up_cap, up_cap['up'], chain['up'])
        self.add(up_cap)
        mb.force_overlap(down_cap, down_cap['up'], chain['down'])
        self.add(down_cap)
        
class AlkaneBox(mb.Compound):
    """A box of alkane chains. """
    def __init__(self, n, n_chains, box_length):
        super(AlkaneBox, self).__init__()
        
        chain = Alkane(n)
        box = mb.Box(lengths=[box_length, box_length, box_length])
        box_of_chains = mb.fill_box(compound=chain, n_compounds=n_chains, box=box)
        self.add(box_of_chains)

To actually use these class definitions to run a simulation, we instantiate the `AlkaneBox` class to yield a `Compound` that contains 200 pentane chains. We then save this to `.gro` and `.top` formats to be able to run a GROMACS simulation. By passing `forcefield_name='oplsaa'` when saving the topology file, we invoke Foyer's atomtyper to apply force field parameters to our system according to the all-atom OPLS force field.

In [ ]:
system = AlkaneBox(n=5, n_chains=200, box_length=4.0)
system.save('system.gro', overwrite=True)
system.save('system.top', forcefield_name='oplsaa', overwrite=True)

Now we have all the necessary files to run a GROMACS simulation. First we'll run a short energy minimization.

In [ ]:
%%bash
gmx grompp -f em.mdp -c system.gro -p system.top -o em.tpr
gmx mdrun -v -deffnm em

Next, we'll perform a 1ns equilibration run.

In [ ]:
%%bash
gmx grompp -f equil.mdp -c em.gro -p system.top -o equil.tpr
gmx mdrun -v -deffnm equil

Finally, we'll perform a sampling run for 10ns.

In [ ]:
%%bash
gmx grompp -f mdp_files/sample.mdp -c equil.gro -p system.top -o sample.tpr
gmx mdrun -v -deffnm sample